In [1]:
!pip install requests beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import json
import csv
import string
import re
import pandas as pd
import os
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime

# turnbackhoax.id

In [ ]:
# Fungsi untuk membersihkan data
def clean_data(text):
    cleaned_text = text.replace("\n", "").replace("\r", "")
    return cleaned_text

# Inisialisasi list untuk menyimpan data
titles = []
contents = []
urls = []

# Loop melalui halaman-halaman
for i in range(1, 541):
    link = requests.get("https://turnbackhoax.id/page/{}".format(i))
    soup = BeautifulSoup(link.text, "html.parser")
    for title, content_url in zip(soup.select('.mh-loop-content h3>a'), soup.select('.mh-loop-content h3>a')):
        title_text = clean_data(title.text.strip())
        content_link = content_url['href']

        # Mengakses halaman konten
        content_page = requests.get(content_link)
        content_soup = BeautifulSoup(content_page.text, "html.parser")
        content_text = clean_data(content_soup.select_one('.entry-content').text.strip())

        titles.append(title_text)
        contents.append(content_text)
        urls.append(content_link)

# Membuat DataFrame
df = pd.DataFrame({"Title": titles, "Content": contents, "URL": urls})

# Membersihkan data
df['Title'] = df['Title'].apply(clean_data)
df['Content'] = df['Content'].apply(clean_data)

# Menambahkan kolom label
df['Label'] = 'HOAX'

# Mendapatkan timestamp saat ini
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Menyimpan DataFrame ke file CSV dengan nama timestamp.csv
filename = f"data_{timestamp}.csv"
df.to_csv(filename, index=False, quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')

print("Data telah disimpan dalam file:", filename)